In [8]:
library(Signac)
library(Seurat)
library(EnsDb.Hsapiens.v86)
library(SeuratDisk)
library(dplyr)
library(BSgenome.Hsapiens.UCSC.hg38)

In [ ]:
ifnb <- readRDS("rbd_processed_rna+atac.rds")

In [ ]:
ifnb <- subset(ifnb, celltype == "microglia")

In [11]:
seqlevels(Annotation(ifnb))

[1] "chrX"  "chr20" "chr1"  "chr6"  "chr3"  "chr7"  "chr12" "chr11" "chr4" 
[10] "chr17" "chr2"  "chr16" "chr8"  "chr19" "chr9"  "chr13" "chr14" "chr5" 
[19] "chr22" "chr10" "chrY"  "chr18" "chr15" "chr21" "chrMT"

In [12]:
DefaultAssay(ifnb) <- "ATAC"
# first compute the GC content for each peak
ifnb <- RegionStats(ifnb, genome = BSgenome.Hsapiens.UCSC.hg38)

Warning message in RegionStats.default(object = regions, genome = genome, verbose = verbose, :
"Not all seqlevels present in supplied genome"


In [ ]:
idents.plot <- c('microglia')
Idents(ifnb) <- idents.plot

### Helpers

In [21]:
library(ggplot2)
save_link_counts <- function(ifnb, gene_list, file_prefix){
    # link peaks to genes
    ifnb <- LinkPeaks(
      object = ifnb,
      peak.assay = "ATAC",
      expression.assay = "SCT",
      genes.use = gene_list
    )

    peak_gene_links <- Links(ifnb[["ATAC"]])
    df <- as.data.frame(peak_gene_links)

    gene_link_counts <- df %>%
      dplyr::filter(pvalue < 0.05) %>%
      group_by(gene) %>%
      summarise(link_count = n())

    write.csv(gene_link_counts, paste0(file_prefix, "gene_link_counts.csv"), row.names = FALSE)
    
    # Return both the original df and gene_link_counts as a list
    return(df)
}

In [22]:
library(ggplot2)
plot_coverage <- function(ifnb, gene_list, file_prefix){
    DefaultAssay(ifnb) <- "ATAC"
    # link peaks to genes
    ifnb <- LinkPeaks(
      object = ifnb,
      peak.assay = "ATAC",
      expression.assay = "SCT",
      genes.use = gene_list
    )

    library(ggplot2)
    for (i in gene_list) { 
        print(i)
        
        # Try running CoveragePlot(), skip if error occurs
        tryCatch({
            p1 <- CoveragePlot(
                object = ifnb,
                region = i,
                features = i,
                expression.assay = "SCT",
                links = TRUE,
                link.assay = "ATAC",
                idents = idents.plot,
                extend.upstream = 50000,
                extend.downstream = 500000
            )
    
            # Save the plot
            ggsave(paste0(file_prefix, i, ".pdf"), p1, width = 15)
        }, error = function(e) {
            message(paste("Skipping:", i, "- Error encountered:", e$message))
        })
    }

}

### Marker Genes; MG0

In [ ]:
# degs RBD vs HC
library(readxl)
library(dplyr)

# Define file path
file_path <- "../../Annotate_Mic_Subtype/11_clusters/Marker_Genes.xlsx"

# Read the specific sheet "Cluster_0"
gene_data <- read_excel(file_path, sheet = "Cluster_0")

# Ensure column names are correctly recognized
colnames(gene_data) <- c("gene", "p_val", "avg_log2FC", "pct.1", "pct.2", "p_val_adj")

# Filter genes based on criteria: p_val_adj < 0.001 and avg_log2FC > 0.25
filtered_genes <- gene_data %>%
  filter(p_val_adj < 0.001, avg_log2FC > 0.25)

# Select the top 50 genes with the smallest p_val_adj (keep original order)
top_genes <- head(filtered_genes$gene, 50)

top_genes

New names:
• `` -> `...1`


[1] "PLXDC2"   "SRGAP2"   "CHST11"   "MEF2A"    "LRMDA"    "EPB41L2" 
 [7] "ITPR2"    "DIAPH2"   "ST6GAL1"  "FOXN3"    "DOCK8"    "ARHGAP26"
[13] "SLC1A3"   "MGAT4A"   "APBB1IP"  "ARHGAP24" "SKAP2"    "FYB1"    
[19] "NCK2"     "SH3RF3"   "FAM49B"   "RNASET2"  "BMP2K"    "RUNX1"   
[25] "DISC1"    "LHFPL2"   "SLC9A9"   "TBXAS1"   "DOCK2"    "DENND3"  
[31] "ATP8B4"   "FGD4"     "PTPRC"    "ETV6"     "GNAQ"     "KCNQ3"   
[37] "SORL1"    "MED13L"   "IPCEF1"   "LDLRAD4"  "ARHGAP15" "SFMBT2"  
[43] "MAML2"    "LNCAROD"  "SSH2"     "FMN1"     "JAZF1"    "FKBP5"   
[49] "TAB2"     "MAML3"

In [ ]:
file_prefix <- "../../Signac_plots/RBD/Marker_Genes_Cluster0/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

save_link_counts(ifnb, top_genes, file_prefix)

Directory already exists: ../Signac_plots/RBD/Marker_Genes_Cluster0/

Testing 50 genes and 43595 peaks

Found gene coordinates for 47 genes

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, GL000009.2, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


gene,link_count
<chr>,<int>
APBB1IP,2
ARHGAP24,1
ARHGAP26,5
BMP2K,2
CHST11,1
DENND3,6
DISC1,2
DOCK2,1
DOCK8,1


In [ ]:
file_prefix <- "../../Signac_plots/RBD/Marker_Genes_Cluster0/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

plot_coverage(ifnb, top_genes, file_prefix)

Directory already exists: ../Signac_plots/RBD/Marker_Genes_Cluster0/



ERROR: Error in plot_coverage(ifnb, top_genes, file_prefix): could not find function "plot_coverage"


### Marker Genes; MG2

In [ ]:
# degs RBD vs HC
library(readxl)
library(dplyr)

# Define file path
file_path <- "../../Annotate_Mic_Subtype/11_clusters/Marker_Genes.xlsx"

# Read the specific sheet "Cluster_0"
gene_data <- read_excel(file_path, sheet = "Cluster_2")

# Ensure column names are correctly recognized
colnames(gene_data) <- c("gene", "p_val", "avg_log2FC", "pct.1", "pct.2", "p_val_adj")

# Filter genes based on criteria: p_val_adj < 0.001 and avg_log2FC > 0.25
filtered_genes <- gene_data %>%
  filter(p_val_adj < 0.001, avg_log2FC > 0.25)

# Select the top 50 genes with the smallest p_val_adj (keep original order)
top_genes <- head(filtered_genes$gene, 50)

top_genes

In [ ]:
file_prefix <- "../../Signac_plots/RBD/Marker_Genes_Cluster2/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

save_link_counts(ifnb, top_genes, file_prefix)

In [ ]:
file_prefix <- "../../Signac_plots/RBD/Marker_Genes_Cluster2/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

plot_coverage(ifnb, top_genes, file_prefix)

Directory already exists: ../Signac_plots/RBD/Marker_Genes_Cluster2/

Testing 27 genes and 43595 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, GL000009.2, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


[1] "FKBP5"


Saving 15 x 6.67 in image
Warning message:
"Removed 4 rows containing missing values or values outside the scale range
(`geom_segment()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "SSH2"


Saving 15 x 6.67 in image
Warning message:
"Removed 170 rows containing missing values or values outside the scale range
(`geom_segment()`)."


...

Saving 15 x 6.67 in image


[1] "RBM47"


Saving 15 x 6.67 in image


[1] "KIF18B"


Saving 15 x 6.67 in image


### Marker Genes; MG3

In [ ]:
# degs RBD vs HC
library(readxl)
library(dplyr)

# Define file path
file_path <- "../../Annotate_Mic_Subtype/11_clusters/Marker_Genes.xlsx"

# Read the specific sheet "Cluster_0"
gene_data <- read_excel(file_path, sheet = "Cluster_3")

# Ensure column names are correctly recognized
colnames(gene_data) <- c("gene", "p_val", "avg_log2FC", "pct.1", "pct.2", "p_val_adj")

# Filter genes based on criteria: p_val_adj < 0.001 and avg_log2FC > 0.25
filtered_genes <- gene_data %>%
  filter(p_val_adj < 0.001, avg_log2FC > 0.25)

# Select the top 50 genes with the smallest p_val_adj (keep original order)
top_genes <- head(filtered_genes$gene, 50)

top_genes

In [ ]:
file_prefix <- "../../Signac_plots/RBD/Marker_Genes_Cluster3/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

save_link_counts(ifnb, top_genes, file_prefix)

In [ ]:
file_prefix <- "../../Signac_plots/RBD/Marker_Genes_Cluster3/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

plot_coverage(ifnb, top_genes, file_prefix)

Directory already exists: ../Signac_plots/RBD/Marker_Genes_Cluster3/

Testing 50 genes and 43595 peaks

Found gene coordinates for 48 genes

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, GL000009.2, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


[1] "IL1RAPL1"


Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Saving 15 x 6.67 in image
Warning message:
"Removed 2 rows containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "ST18"


Saving 15 x 6.67 in image


...

Saving 15 x 6.67 in image
Warning message:
"Removed 69 rows containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "AC008571.2"


Skipping: AC008571.2 - Error encountered: Gene not found



[1] "HHIP"


Saving 15 x 6.67 in image


### DEGs RBD vs HC; MG0

In [ ]:
# update plot_coverage function
library(Signac)
library(patchwork)   # lets us manipulate the assembled plot
library(ggplot2)

plot_coverage <- function(ifnb, gene_list, file_prefix,
                          idents.plot = NULL,
                          extend.up = 5e4, extend.down = 5e5) {

  DefaultAssay(ifnb) <- "ATAC"
  ifnb <- LinkPeaks(ifnb,
                    peak.assay       = "ATAC",
                    expression.assay = "SCT",
                    genes.use        = gene_list)

  for (g in gene_list) {
    message("Plotting ", g)

    tryCatch({

      cov <- CoveragePlot(
             ifnb,
             region            = g,
             features          = g,
             expression.assay  = "SCT",
             links             = FALSE,
             link.assay        = "ATAC",
             idents            = idents.plot,
             extend.upstream   = extend.up,
             extend.downstream = extend.down
           )
       arc  <- LinkPlot(ifnb, region = g, assay = "ATAC",
                  extend.upstream = 5e4, extend.downstream = 5e5) +
       scale_colour_gradient2(low = "blue", mid = "grey",
                               high = "#ca0020", midpoint = 0,
                               limits = c(-0.1, 0.1), name = "Link\nscore") +
        guides(colour = guide_colourbar(title.position = "top")) +
        theme(legend.position = "right")

       final <- cov / arc

       ggsave(paste0(file_prefix, g, ".pdf"), final, width = 15, height = 6)

    }, error = function(e) {
      message("Skipping ", g, ": ", e$message)
    })
  }
}

In [ ]:
# degs RBD vs HC
library(readxl)
library(dplyr)

# Define file path
file_path <- "../../DEG_GO_Analysis/DEGs_RBD_vs_HC/Significant_DEGs_RBD_vs_HC.xlsx"

# Read the specific sheet "Cluster_0"
gene_data <- read_excel(file_path, sheet = "Cluster_0")

# Ensure column names are correctly recognized
colnames(gene_data) <- c("gene", "p_val", "avg_log2FC", "pct.1", "pct.2", "p_val_adj")

# Filter genes based on criteria: p_val_adj < 0.001 and avg_log2FC > 0.25
filtered_genes <- gene_data %>%
  filter(p_val_adj < 0.001, avg_log2FC > 0.25)

# Select the top 50 genes with the smallest p_val_adj (keep original order)
top_genes <- head(filtered_genes$gene, 50)

top_genes

Warning message:
“package ‘readxl’ was built under R version 4.3.3”
Warning message:
“package ‘dplyr’ was built under R version 4.3.3”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


New names:
• `` -> `...1`


[1] "DPYD"      "TMEM163"   "PVT1"      "TTTY14"    "TPRG1"     "USP9Y"    
 [7] "LINC00278" "BACH1"     "TBC1D8"    "BCL6"      "PRKY"      "SCFD2"    
[13] "NIBAN1"    "PTPN2"     "SH3RF3"    "TBC1D14"   "SPATA6"    "DENND3"   
[19] "NLGN4Y"    "SLC11A1"   "CD163"     "FLT1"      "ACER3"     "KCNIP4"   
[25] "SCIN"      "FMNL2"     "ANK2"      "IARS"      "FCGBP"     "SPP1"     
[31] "FRMD4B"    "IL21R"     "SND1"      "ASAP1"     "ACSL1"     "WARS"     
[37] "RPS4Y1"    "GRB2"      "ADGRE2"    "ARHGAP20"  "PTPRE"     "KCNQ3"    
[43] "DSE"       "FNDC3B"    "ARHGAP26"  "LINC01146" "ZFY"       "LIMK2"    
[49] "GAS7"      "PXK"

In [ ]:
# # get union of DEGs from RBD and PD vs HC
# deg_rbd = read.csv("../../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster0/gene_link_counts.csv")$gene
# deg_rbd

[1] "ACER3"     "ACSL1"     "ADGRE2"    "ANK2"      "ARHGAP20"  "ARHGAP26" 
 [7] "ASAP1"     "BACH1"     "BCL6"      "CD163"     "DENND3"    "DPYD"     
[13] "FCGBP"     "FLT1"      "FMNL2"     "FNDC3B"    "GAS7"      "GRB2"     
[19] "IARS"      "KCNQ3"     "LIMK2"     "LINC00278" "PTPN2"     "PTPRE"    
[25] "PVT1"      "PXK"       "RPS4Y1"    "SCIN"      "SH3RF3"    "SLC11A1"  
[31] "SND1"      "SPATA6"    "SPP1"      "TBC1D14"   "TBC1D8"    "TMEM163"  
[37] "TPRG1"     "TTTY14"    "USP9Y"     "ZFY"

In [ ]:
# deg_pd = read.csv("../../Signac_plots/PD/DEGs_PD_vs_HC_Cluster0/gene_link_counts.csv")$gene
# deg_pd

[1] "CARD11" "DDX60L" "DLEU1"  "DLEU7"  "HECTD2" "MGAT4A" "MYCBP2" "TANC1" 
[9] "TTC7A"

In [ ]:
# deg = c(deg_rbd, deg_pd)
# deg --> saved to "../../../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster0/gene_link_counts.csv"; can be read in directly

[1] "ACER3"     "ACSL1"     "ADGRE2"    "ANK2"      "ARHGAP20"  "ARHGAP26" 
 [7] "ASAP1"     "BACH1"     "BCL6"      "CD163"     "DENND3"    "DPYD"     
[13] "FCGBP"     "FLT1"      "FMNL2"     "FNDC3B"    "GAS7"      "GRB2"     
[19] "IARS"      "KCNQ3"     "LIMK2"     "LINC00278" "PTPN2"     "PTPRE"    
[25] "PVT1"      "PXK"       "RPS4Y1"    "SCIN"      "SH3RF3"    "SLC11A1"  
[31] "SND1"      "SPATA6"    "SPP1"      "TBC1D14"   "TBC1D8"    "TMEM163"  
[37] "TPRG1"     "TTTY14"    "USP9Y"     "ZFY"       "CARD11"    "DDX60L"   
[43] "DLEU1"     "DLEU7"     "HECTD2"    "MGAT4A"    "MYCBP2"    "TANC1"    
[49] "TTC7A"

In [ ]:
deg = read.csv("../../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster0/gene_link_counts.csv")$gene

In [ ]:
file_prefix <- "../../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster0/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

# save_link_counts(ifnb, top_genes, file_prefix)
gene_link_counts <- save_link_counts(ifnb, deg, file_prefix)

Directory already exists: ../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster0/

Testing 49 genes and 43595 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, GL000009.2, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


gene,link_count
<chr>,<int>
ACER3,1
ACSL1,3
ADGRE2,4
ARHGAP20,2
ARHGAP26,5
ASAP1,1
BACH1,3
BCL6,2
CARD11,4


In [ ]:
file_prefix <- "../../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster0/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

# plot_coverage(ifnb, top_genes, file_prefix)
plot_coverage(ifnb, deg, file_prefix)

Directory already exists: ../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster0/

Testing 47 genes and 43595 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, GL000009.2, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."
Plotting ACER3

Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.
Warning message:
"Removed 53 rows containing missing values or values outside the scale range
(`geom_segment()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`)."
Plotting ACSL1

Scale for colour is already present.
Adding another scale for colour, 

In [23]:
plot_coverage(ifnb, "GRB2", file_prefix)

Testing 1 genes and 43595 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, GL000009.2, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."
Plotting GRB2

Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.
Warning message:
"Removed 61 rows containing missing values or values outside the scale range
(`geom_segment()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`)."


In [25]:
plot_coverage(ifnb, "TMEM163", file_prefix)

Testing 1 genes and 43595 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, GL000009.2, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."
Plotting TMEM163

Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.
Warning message:
"Removed 125 rows containing missing values or values outside the scale range
(`geom_segment()`)."


### DEGs RBD vs HC; MG2

In [ ]:
# Define file path
file_path <- "../../DEG_GO_Analysis/DEGs_RBD_vs_HC/Significant_DEGs_RBD_vs_HC.xlsx"

# Read the specific sheet "Cluster_0"
gene_data <- read_excel(file_path, sheet = "Cluster_2")

# Ensure column names are correctly recognized
colnames(gene_data) <- c("gene", "p_val", "avg_log2FC", "pct.1", "pct.2", "p_val_adj")

# Filter genes based on criteria: p_val_adj < 0.001 and avg_log2FC > 0.25
filtered_genes <- gene_data %>%
  filter(p_val_adj < 0.001, avg_log2FC > 0.25)

# Select the top 50 genes with the smallest p_val_adj (keep original order)
top_genes <- head(filtered_genes$gene, 50)

top_genes

In [ ]:
deg_rbd = read.csv("../../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster2/gene_link_counts.csv")$gene
deg_pd = read.csv("../../Signac_plots/PD/DEGs_PD_vs_HC_Cluster2/gene_link_counts.csv")$gene
deg = c(deg_rbd, deg_pd)

In [ ]:
file_prefix <- "../../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster2/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

save_link_counts(ifnb, deg, file_prefix)

Directory already exists: ../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster2/

Testing 45 genes and 43595 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, GL000009.2, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


gene,link_count
<chr>,<int>
ACER3,1
ADGRE2,4
ARHGAP26,5
ARID5B,3
BACH1,3
BCL6,2
CD163,2
CD163L1,2
CIT,3


In [ ]:
file_prefix <- "../../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster2/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

# plot_coverage(ifnb, top_genes, file_prefix)
plot_coverage(ifnb, deg, file_prefix)

Directory already exists: ../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster2/

Testing 45 genes and 43595 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, GL000009.2, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


[1] "ACER3"


Saving 15 x 6.67 in image
Warning message:
"Removed 53 rows containing missing values or values outside the scale range
(`geom_segment()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "ADGRE2"


Saving 15 x 6.67 in image
Warning message:
"Removed 67 rows containing missing values or values outside the scale range
(`geom_segment()`)."


...

Saving 15 x 6.67 in image
Warning message:
"Removed 22 rows containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "UBTD2"


Saving 15 x 6.67 in image
Warning message:
"Removed 63 rows containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "XAF1"


Saving 15 x 6.67 in image
Warning message:
"Removed 55 rows containing missing values or values outside the scale range
(`geom_segment()`)."


### DEGs RBD vs HC; MG3

In [ ]:
# Define file path
file_path <- "../../DEG_GO_Analysis/DEGs_RBD_vs_HC/Significant_DEGs_RBD_vs_HC.xlsx"

# Read the specific sheet "Cluster_0"
gene_data <- read_excel(file_path, sheet = "Cluster_3")

# Ensure column names are correctly recognized
colnames(gene_data) <- c("gene", "p_val", "avg_log2FC", "pct.1", "pct.2", "p_val_adj")

# Filter genes based on criteria: p_val_adj < 0.001 and avg_log2FC > 0.25
filtered_genes <- gene_data %>%
  filter(p_val_adj < 0.001, avg_log2FC > 0.25)

# Select the top 50 genes with the smallest p_val_adj (keep original order)
top_genes <- head(filtered_genes$gene, 50)

top_genes

In [ ]:
deg_rbd = read.csv("../../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster3/gene_link_counts.csv")$gene
deg_pd = read.csv("../../Signac_plots/PD/DEGs_PD_vs_HC_Cluster3/gene_link_counts.csv")$gene
deg = c(deg_rbd, deg_pd)

In [ ]:
deg = read.csv("../../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster3/gene_link_counts.csv")$gene

In [ ]:
file_prefix <- "../../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster3/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

df = save_link_counts(ifnb, deg, file_prefix)

Directory already exists: ../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster3/

Testing 6 genes and 43595 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, GL000009.2, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


In [14]:
df %>% filter(gene == "PLCG2")

seqnames,start,end,width,strand,score,gene,peak,zscore,pvalue
<fct>,<int>,<int>,<int>,<fct>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
chr16,81470244,81739097,268854,*,-0.10189926,PLCG2,chr16-81469701-81470787,-1.963549,0.02479120
chr16,81479085,81739097,260013,*,-0.11348874,PLCG2,chr16-81478600-81479569,-2.093330,0.01815984
chr16,81481493,81739097,257605,*,-0.10252591,PLCG2,chr16-81480990-81481995,-1.955862,0.02524069
chr16,81738188,81739097,910,*,0.05174534,PLCG2,chr16-81737680-81738696,1.649895,0.04948226
chr16,81739097,82190436,451340,*,-0.09733341,PLCG2,chr16-82189967-82190905,-2.158557,0.01544227


In [ ]:
file_prefix <- "../../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster3/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

plot_coverage(ifnb, deg, file_prefix)

Directory already exists: ../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster3/

Testing 6 genes and 43595 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, GL000009.2, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


[1] "ADAMTS18"


Saving 15 x 6.67 in image
Warning message:
"Removed 36 rows containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "GPM6A"


Saving 15 x 6.67 in image


...

Saving 15 x 6.67 in image
Warning message:
"Removed 59 rows containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "CLECL1"


Saving 15 x 6.67 in image
Warning message:
"Removed 57 rows containing missing values or values outside the scale range
(`geom_segment()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "TRIM26"


Saving 15 x 6.67 in image
Warning message:
"Removed 19 rows containing missing values or values outside the scale range
(`geom_segment()`)."


In [ ]:
file_prefix <- "../../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster3/"
plot_coverage(ifnb, "PLCG2", file_prefix)

Testing 1 genes and 43595 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, GL000009.2, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."
Plotting PLCG2

Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.
Warning message:
"Removed 3 rows containing missing values or values outside the scale range
(`geom_segment()`)."


### DAM

In [23]:
dam_ad_genes <- c("CD83", "NAMPT", "HIF1A", "SRGN", "FTH1", "FTL", "RPL32", "TPT1", "RPLP1", 
               "RPS4X", "RPS24", "RPS11", "RPS15", "PLEK", "CCL3", "IL1B", "IFIT2", "B2M", 
               "CD74", "MAMDC2", "EEF1B2", "PDPN", "RPL10A", "RPL34", "RPL35A", "RPL37", 
               "RPL7", "RPL38", "RPS16", "RPL6", "RPS14", "RPS9", "RPL30", "APOE", "RPS25", 
               "RPS20", "EEF1A1", "RPS8", "RPL13", "RPL14", "RPL8", "RPL35", "RPL23", "FAU", 
               "RPS19", "RPL27A", "RPS3", "RPL37A", "ELOVL5", "SOCS6", "CADM1", 
               "ITGAX", "NAP1L1", "CEBPA", "TREM2", "PLXDC2", "SOAT1", "LIPA", "CD84", "CREG1", 
               "FAM20C", "SLC16A3", "ARHGEF7", "GPI", "CUX1", "DHRS3", "CSF2RA", "PSAP", 
               "RASSF3", "RHEB", "BCL2", "NFE2L2", "TLR2", "CHPT1", "FGL2", "EFR3A", "ARAP2", 
               "CLEC7A", "CYBA", "ITM2B", "CD37", "ITGB2", "ARPC3", "HLA-E", "EEF2", "PABPC1", 
               "NPC2", "FCER1G", "RPSA", "TYROBP", "RPL31", "EEF1D", "RPL12", "CHST11", "FCGR3A", 
               "AZIN1", "MFSD1", "GNAS", "NOP58", "LAT2", "CTSZ", "SNX3", "ADAR", "AXL", "BST2", 
               "OXCT1", "SREBF2", "NRP1", "USP12", "GPNMB", "MYO1E", "SPP1", "ELL2", "PADI2", 
               "ATP1B3", "RNF149", "SLC11A1", "KLHL6", "PGK1", "CTSB", "SDCBP")

In [ ]:
file_prefix <- "../../Signac_plots/RBD/DAM_AD/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

save_link_counts(ifnb, dam_ad_genes, file_prefix)

Directory already exists: ../Signac_plots/RBD/DAM_AD/

Testing 114 genes and 43595 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, GL000009.2, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


seqnames,start,end,width,strand,score,gene,peak,zscore,pvalue
<fct>,<int>,<int>,<int>,<fct>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
chr1,12143785,12617731,473947,*,-0.06273120,DHRS3,chr1-12143256-12144313,-1.979004,2.390780e-02
chr1,12175242,12617731,442490,*,-0.06431960,DHRS3,chr1-12174754-12175729,-2.093986,1.813062e-02
chr1,12407259,12617731,210473,*,0.06826675,DHRS3,chr1-12406385-12408132,2.603021,4.620308e-03
chr1,12496646,12617731,121086,*,0.07815146,DHRS3,chr1-12496177-12497115,3.135552,8.576536e-04
chr1,12525641,12617731,92091,*,0.06446448,DHRS3,chr1-12525127-12526154,2.325748,1.001601e-02
chr1,13583465,13623507,40043,*,0.05627361,PDPN,chr1-13623179-13623834,1.877066,3.025450e-02
chr1,16905012,17119435,214424,*,0.06018502,PADI2,chr1-16904553-16905470,2.031767,2.108860e-02
chr1,17059921,17119435,59515,*,0.05557785,PADI2,chr1-17059467-17060374,1.944377,2.592499e-02
chr1,17119435,17119565,131,*,0.05294900,PADI2,chr1-17119125-17120005,1.825200,3.398544e-02


In [ ]:
file_prefix <- "../../Signac_plots/RBD/DAM_AD/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

plot_coverage(ifnb, dam_ad_genes, file_prefix)

Directory already exists: ../Signac_plots/RBD/DAM_AD/

Testing 114 genes and 43595 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, GL000009.2, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


[1] "CD83"


Saving 15 x 6.67 in image


[1] "NAMPT"


Saving 15 x 6.67 in image
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`)."


...

Saving 15 x 6.67 in image
Warning message:
"Removed 52 rows containing missing values or values outside the scale range
(`geom_segment()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "NAP1L1"


Saving 15 x 6.67 in image
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "CEBPA"


Saving 15 x 6.67 in image
Warning message:
"Removed 19 rows containing missing values or values outside the scale range
(`geom_segment()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`)."
